## Chatbot -> Remember previous interactions

-> Using GROQ api to call the models

In [ ]:
from dotenv import load_dotenv
import os
load_dotenv() 

# langsmith tracking
#https://smith.langchain.com/
# os.environ['LANGCHAIN_API_KEY']=os.getenv("LANGCHAIN_API_KEY")
# os.environ['LANGCHAIN_TRACING_V2'] = 'true'
# os.environ['LANGCHAIN_PROJECT'] = os.getenv("LANGCHAIN_PROJECT")

os.environ['GROQ_API_KEY'] = os.getenv('GROQ_API_KEY')
#groqAPIkey = os.getenv('GROQ_API_KEY')

In [2]:
from langchain_groq import ChatGroq
llm=ChatGroq(model='Gemma2-9b-It')

In [3]:
from langchain_core.messages import HumanMessage

llm.invoke([HumanMessage(content="Hi, I am Subodh, working as a data scientist")])

AIMessage(content="Hello Subodh!\n\nIt's great to meet you.  \n\nWhat kind of data science work do you do? Are you working on any interesting projects right now? \n\nI'm always eager to learn more about how people are using data science to solve problems and make a difference.\n", response_metadata={'token_usage': {'completion_tokens': 64, 'prompt_tokens': 22, 'total_tokens': 86, 'completion_time': 0.116363636, 'prompt_time': 0.000191129, 'queue_time': 0.014145101, 'total_time': 0.116554765}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-ce35d1c0-f02b-443b-9e41-c4e79ce80c38-0', usage_metadata={'input_tokens': 22, 'output_tokens': 64, 'total_tokens': 86})

In [5]:
llm.invoke("What is my name? i told you ") # here conversation history does not know to LLM

# Model does not have no memory of past conversations. 
# Every interaction we have is a fresh start.\n\nIf you'd like to tell me your name, I'm happy to remember it for this conversation! 😄  \n\n", response_metadata={'token_usage': {'completion_tokens': 52, 'prompt_tokens': 23, 'total_tokens': 75, 'completion_time': 0.104774858, 'prompt_time': 0.002321479, 'queue_time': None, 'total_time': 0.107096337}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-3837de9a-3d76-4765-a824-8bb03177d3b2-0', usage_metadata={'input_tokens': 23, 'output_tokens': 52, 'total_tokens': 75})

AIMessage(content="As a large language model, I have no memory of past conversations. If you'd like to tell me your name, I'd be happy to know! 😊  \n\n", response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 23, 'total_tokens': 62, 'completion_time': 0.079614143, 'prompt_time': 0.002610718, 'queue_time': None, 'total_time': 0.082224861}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-4e5a8f45-1cdc-4994-83b3-c3de8cd0c721-0', usage_metadata={'input_tokens': 23, 'output_tokens': 39, 'total_tokens': 62})

In [1]:
from langchain_core.messages import AIMessage
llm.invoke([
        HumanMessage(content="Hi, I am Subodh, working as a data scientist"),
        AIMessage(content="Hi Subodh!\n\nIt's great to meet you.  \n\nWhat kind of data science work do you do?  I'm always interested in learning more about how people are using AI and data analysis in different fields.")
       , HumanMessage(content="Hey, What is my name and what do i do?")
    ]
)

NameError: name 'llm' is not defined

## Message History
-> session: for particular user we maintain session. 

In current interaction with LLM, it will load the previos interactions and

then send the current input + history to LLM


RunnableWithMessageHistory: A class that runs an LLM (Language Model) while maintaining message history, allowing for continuous conversation.

In [6]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# define LLM
from langchain_groq import ChatGroq
llm=ChatGroq(model='Gemma2-9b-It')

# this fuction take session id as input, it gives the history of that chat and returns it
# means = for specifice session id what is the chat history present. 
store = {}
def getSessionhistory(sessionId:str)->BaseChatMessageHistory: # return type for manage structure of history
    if sessionId not in store:
        store[sessionId] = ChatMessageHistory()
    return store[sessionId]


with_messageHistory = RunnableWithMessageHistory(llm, getSessionhistory)

config = {"configurable":{"session_id":"chat1"}} # it contain at what session id 


In [7]:
response =with_messageHistory.invoke(
    [HumanMessage(content="hi, i am Subodh, I am working in wipro as a data scientist")],
    config=config
)

In [8]:
response

AIMessage(content="Hello Subodh,\n\nThat's great to hear! It's fantastic that you're working as a Data Scientist at Wipro.  \n\nData science is a rapidly growing field with lots of exciting opportunities. \n\nWhat kind of projects are you working on at the moment? Are there any particular areas of data science that you're particularly interested in? \n\nI'd love to hear more about your work!\n", response_metadata={'token_usage': {'completion_tokens': 91, 'prompt_tokens': 27, 'total_tokens': 118, 'completion_time': 0.165454545, 'prompt_time': 0.000156859, 'queue_time': 0.012960900999999999, 'total_time': 0.165611404}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-780ed92f-5df1-4579-9da0-afab37815d26-0', usage_metadata={'input_tokens': 27, 'output_tokens': 91, 'total_tokens': 118})

In [9]:
store

{'chat1': InMemoryChatMessageHistory(messages=[HumanMessage(content='hi, i am Subodh, I am working in wipro as a data scientist'), AIMessage(content="Hello Subodh,\n\nThat's great to hear! It's fantastic that you're working as a Data Scientist at Wipro.  \n\nData science is a rapidly growing field with lots of exciting opportunities. \n\nWhat kind of projects are you working on at the moment? Are there any particular areas of data science that you're particularly interested in? \n\nI'd love to hear more about your work!\n", response_metadata={'token_usage': {'completion_tokens': 91, 'prompt_tokens': 27, 'total_tokens': 118, 'completion_time': 0.165454545, 'prompt_time': 0.000156859, 'queue_time': 0.012960900999999999, 'total_time': 0.165611404}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-780ed92f-5df1-4579-9da0-afab37815d26-0', usage_metadata={'input_tokens': 27, 'output_tokens': 91, 'total_tokens': 118})])}

In [ ]:
def ask_Que(user):
    response=with_messageHistory.invoke([
                HumanMessage(content=user)
            ] , config=config)
    print(response) 
ask_Que("What is my name")

# invoke: This method sends a message to the LLM and processes it within the context of the session. 

content='Your name is Subodh. \n\nYou told me at the beginning of our conversation!  😊  \n\nIs there anything else I can help you with?\n' response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 130, 'total_tokens': 166, 'completion_time': 0.065454545, 'prompt_time': 0.003587151, 'queue_time': 0.009835809, 'total_time': 0.069041696}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None} id='run-055a9082-687d-49a2-818d-bf71735bec1f-0' usage_metadata={'input_tokens': 130, 'output_tokens': 36, 'total_tokens': 166}


In [11]:
store

{'chat1': InMemoryChatMessageHistory(messages=[HumanMessage(content='hi, i am Subodh, I am working in wipro as a data scientist'), AIMessage(content="Hello Subodh,\n\nThat's great to hear! It's fantastic that you're working as a Data Scientist at Wipro.  \n\nData science is a rapidly growing field with lots of exciting opportunities. \n\nWhat kind of projects are you working on at the moment? Are there any particular areas of data science that you're particularly interested in? \n\nI'd love to hear more about your work!\n", response_metadata={'token_usage': {'completion_tokens': 91, 'prompt_tokens': 27, 'total_tokens': 118, 'completion_time': 0.165454545, 'prompt_time': 0.000156859, 'queue_time': 0.012960900999999999, 'total_time': 0.165611404}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-780ed92f-5df1-4579-9da0-afab37815d26-0', usage_metadata={'input_tokens': 27, 'output_tokens': 91, 'total_tokens': 118}), Hu

In [12]:
def ask_Que(user):
    response=with_messageHistory.invoke([
                HumanMessage(content=user)
            ] , config=config)
    print(response)
ask_Que("What i am doing?")

content="You are chatting with me, an AI assistant! \n\nYou told me you are Subodh and that you work as a Data Scientist at Wipro. \n\nIs there anything specific you'd like to talk about or ask me? I'm here to help! 😊 \n" response_metadata={'token_usage': {'completion_tokens': 61, 'prompt_tokens': 179, 'total_tokens': 240, 'completion_time': 0.110909091, 'prompt_time': 0.006902584, 'queue_time': 0.0069755749999999995, 'total_time': 0.117811675}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None} id='run-8e91d15f-943e-4f29-a044-fdff3d79e44e-0' usage_metadata={'input_tokens': 179, 'output_tokens': 61, 'total_tokens': 240}


In [13]:
store

{'chat1': InMemoryChatMessageHistory(messages=[HumanMessage(content='hi, i am Subodh, I am working in wipro as a data scientist'), AIMessage(content="Hello Subodh,\n\nThat's great to hear! It's fantastic that you're working as a Data Scientist at Wipro.  \n\nData science is a rapidly growing field with lots of exciting opportunities. \n\nWhat kind of projects are you working on at the moment? Are there any particular areas of data science that you're particularly interested in? \n\nI'd love to hear more about your work!\n", response_metadata={'token_usage': {'completion_tokens': 91, 'prompt_tokens': 27, 'total_tokens': 118, 'completion_time': 0.165454545, 'prompt_time': 0.000156859, 'queue_time': 0.012960900999999999, 'total_time': 0.165611404}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-780ed92f-5df1-4579-9da0-afab37815d26-0', usage_metadata={'input_tokens': 27, 'output_tokens': 91, 'total_tokens': 118}), Hu

## Prompt Template

In [14]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage,SystemMessage
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory


prompt = ChatPromptTemplate.from_messages([

("system","You are a helpful assistant. Answer all the question to the best of your ability in {language} language.  "),
MessagesPlaceholder(variable_name="messages")

])

chain = prompt|llm

In [17]:
chain.invoke({"messages":[HumanMessage(content="Hi my name is Subodh")] , 
              "language":"Hindi"})
#chain.invoke({"messages":["Hi my name is Subodh" ], 
#              "language":"Hindi"})

AIMessage(content='नमस्ते Subodh,  मैं आपकी मदद करने के लिए यहाँ हूँ। \n\nआप मुझसे कोई भी प्रश्न पूछ सकते हैं और मैं अपनी पूरी क्षमता के अनुसार उत्तर देने की कोशिश करूँगा। \n\n', response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 37, 'total_tokens': 96, 'completion_time': 0.107272727, 'prompt_time': 0.000395548, 'queue_time': 0.012831351999999999, 'total_time': 0.107668275}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-f5026073-a2e5-4d53-b366-520aeab88cdc-0', usage_metadata={'input_tokens': 37, 'output_tokens': 59, 'total_tokens': 96})

In [18]:
withMessageHistory = RunnableWithMessageHistory(chain, getSessionhistory, input_messages_key="messages") # chain is send as a input

In [19]:
config = {"configurable":{"session_id":"chat2"}} 

In [20]:
response = withMessageHistory.invoke(
    {"messages": [HumanMessage(content="Hi my name is Subodh")], 
     "language": "hindi"}, 
    config=config
)
response.content


'नमस्ते Subodh! 😊  \n\nमुझे आपसे मिलकर खुशी हो रही है।  \nआप मुझसे क्या पूछना चाहेंगे? \n'

In [21]:
response.content

'नमस्ते Subodh! 😊  \n\nमुझे आपसे मिलकर खुशी हो रही है।  \nआप मुझसे क्या पूछना चाहेंगे? \n'

In [22]:
response = withMessageHistory.invoke(
    {"messages": [HumanMessage(content="What is my name")], 
     "language": "hindi"}, 
    config=config
)
response.content

'आपका नाम Subodh है।  \n'

#### Sometimes history becomes very long 

Manage the conversation history

trip messages => helps to reduce how many messages we are sending to model. 


In [25]:
from langchain_core.messages import trim_messages, AIMessage,HumanMessage, SystemMessage

trimmer = trim_messages(
    max_tokens = 35,
    strategy = "last",
    token_counter = llm , # genai model
    include_system = True,
    allow_partial = False,
    start_on = 'human'
)

messages = [
    SystemMessage(content = "You are the good AI assistant."), 

    HumanMessage(content = "Hi, I am a bod"),
    AIMessage(content="Hi!"),
    HumanMessage(content="I like vanilla ice-cream"),
    AIMessage(content="Nice"),
    HumanMessage(content="What is 2+2"),
    AIMessage(content="ans is 4"),
    HumanMessage(content="Thanks"),
    AIMessage(content="No Problem"),
    HumanMessage(content="Having Fun"),
    AIMessage(content="Yes")
]

trimmer.invoke(messages)

OSError: [WinError 126] The specified module could not be found. Error loading "c:\Users\Admin\Desktop\GenAI Krishna naik\Langchain Basics\.venv\Lib\site-packages\torch\lib\fbgemm.dll" or one of its dependencies.

In [24]:
!pip install ipywidgets

   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   ---------------------------------------- 2.3/2.3 MB 16.6 MB/s eta 0:00:00


In [65]:
!pip install torch